# Sensitivity Analysis

In this tutorial we demonstrate how to perform sensitivity analysis as part of the `AutoEmulate` workflow. The tutorial covers:

1. Setting up an example simulation: here we use our "FlowProblem" simulator. The system simulated is from the field of cardiovascular modeling, a tube with an input flow rate at any given time. The tube is divided to 10 compartments which allows for the study of the pressure and flow rate at various points in the tube. See "The Flow Problem" below for more details.
2. Running the simulation for 100 sets of parameters sampled from the parameter space.
3. Using Autoemulate to find the best emulator for this simulation
4. Performing sensitivity analysis.

<details>
<summary>The Flow Problem</summary>

In the field of cardiovascular modeling, capturing the dynamics of blood flow and the associated pressures and volumes within the vascular system is crucial for understanding heart function and disease. Physics-based models that accurately represent these dynamics often require significant computational resources, making them challenging to apply in large-scale or real-time scenarios. Emulation techniques provide a way to achieve high-fidelity simulations of the cardiovascular system, allowing for efficient and accurate analysis of key hemodynamic parameters.
</details>

In [ ]:
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.simulations.flow_problem import FlowProblem
import warnings
warnings.filterwarnings("ignore")
figsize = (9, 5)

Set up the simulation parameters and ranges:

In [ ]:
parameters_range = {"T": (0.5, 2.0), # Cardiac cycle period (s)
                "td": (0.1, 0.5), # Pulse duration (s)
                "amp": (100.0, 1000.0), # Amplitude (e.g., pressure or flow rate)
                "dt": (0.0001, 0.01), # Time step (s)
                "C": (20.0, 60.0), # Compliance (unit varies based on context)
                "R": (0.01, 0.1), # Resistance (unit varies based on context)
                "L": (0.001, 0.005), # Inductance (unit varies based on context)
                "R_o": (0.01, 0.05), # Outflow resistance (unit varies based on context)
                "p_o": (5.0, 15.0)} # Initial pressure (unit varies based on context)
output_names = ["pressure"]

simulator = FlowProblem(
    parameters_range=parameters_range,
    output_names=output_names,
)

Run the simulation for 100 sets of parameters sampled from the parameter space:

In [ ]:
x = simulator.sample_inputs(100)
y = simulator.forward_batch(x)

In [ ]:
x.shape, y.shape

Use AutoEmulate to find the best emulator for this simulation:

In [ ]:
ae = AutoEmulate(x, y, models=["MLP", "GP"])  # remove models argument to use all models
best = ae.best_result()
best.model_name

### Sensitivity Analysis

1. Define the problem by creating a dictionary which contains the names and the boundaries of the parameters 
2. Evaluate the contribution of each parameter via the Sobol and Morris methods.

In [ ]:
problem = {
    'num_vars': simulator.in_dim,
    'names': simulator.param_names,
    'bounds': simulator.param_bounds,
    'output_names': simulator.output_names,
    }
sa = SensitivityAnalysis(best.model, problem=problem)

Sobol metrics:

- $S_1$: First-order sensitivity index.
- $S_2$: Second-order sensitivity index.
- $S_t$: Total sensitivity index.

Sobol interpretation:
- $S_1$ values sum to ≤ 1.0 (exact fraction of variance explained)
- $S_t - S_1$ = interaction effects involving that parameter
- Large $S_t - S_1$ gap indicates strong interactions


In [ ]:
sobol_df = sa.run("sobol")
sobol_df

In [ ]:
sa.plot_sobol(sobol_df, index="ST", figsize=figsize) 

Morris Interpretation:

- High $\mu^*$, Low $\sigma$: Important parameter with linear/monotonic effects
- High $\mu^*$, High $\sigma$: Important parameter with non-linear effects or interactions
- Low $\mu^*$, High $\sigma$: Parameter involved in interactions but not individually important
- Low $\mu^*$, Low $\sigma$: Unimportant parameter

In [ ]:
morris_df = sa.run("morris")
morris_df

In [ ]:
sa.plot_morris(morris_df, figsize=figsize)